<a href="https://colab.research.google.com/github/wassimchouchen/Automatic-Speech-Recognition-/blob/main/Identify_medical_vocabulary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
!pip install pydub
import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding( 
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib
import pydub

from sklearn.cluster import AgglomerativeClustering
import numpy as np
import torch

# Check if a GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
num_speakers = 2 
language = 'any' 
model_size = 'medium'

In [3]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving conv.mp3 to conv.mp3


In [4]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'
# Load the audio file
audio_file = pydub.AudioSegment.from_wav(path)
audio_file = audio_file.set_channels(1)
audio_file.export(path, format="wav")
with wave.open(path) as f:
  print(f.getnchannels())

1


In [5]:
model = whisper.load_model(model_size)

In [6]:
result = model.transcribe(path)
segments = result["segments"]

In [7]:
text=result["text"]

In [8]:
text

" podcastfrancaisfacile.com Bonjour. Bonjour, installez-vous, je vous en prie. Merci. Alors, qu'est-ce qui vous amène ? Eh bien, j'ai des douleurs au ventre, au niveau de l'estomac. Vous avez mal à l'estomac ? En fait, je ne suis pas sûr que ce soit à l'estomac. Vous avez mal en permanence ? Non, ce n'est pas tout le temps, c'est surtout après manger. Comment est la douleur ? Ce n'était pas très fort au début, mais depuis quelques jours, c'est très aigu. Quand ont commencé les premières douleurs ? C'était il y a deux mois environ. Je pensais que ça passerait, mais finalement, c'est de pire en pire. Bien, on va regarder ça. Enlevez votre chemise et allongez-vous."

In [9]:
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")

In [11]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [12]:
sequence_to_classify = text
candidate_labels = ['anatomie', 'pelvien', "périnée", "vagin", "vaginal", "anus", "rectal",
"rectum", "utérus", "utérin", "urètre", "cavité", "endocavitaire" , "examens", "touché vaginale", 
"touché mammaire", "touché abdominal", "touché rectale", "échographie", "frottis", "prélèvement" ,
"instruments médicaux","speculum", "sondes", "gants", "lubrifiant" , "pathologies" , "HPV", "IST", "MST", "incontinence",
"endométriose", "douleur", "périnéale"]
result=classifier(sequence_to_classify, candidate_labels, multi_class=True)
for i,j in zip(result["labels"],result["scores"]):
  print(i,":",j)